## Harvard's Expansion
A continuation/complication of the physical impact document, this document will outline how Harvard has changed over the past 2 decades, starting in the year 2000 and ending in the year 2016.

The majority of the data for this expansion was pulled from: http://www.cambridgema.gov/CDD/planud/institutionalplanning/oldertowngownreports.

The "change" will focus on Cambridge, where the data is most readily available and complete. Future projects may focus on other key places for Harvard's expansion, like Allston. Harvard's change is found in its taxable and tax-exempt property growth, its student population growth, and the taxes and PILOT (Payment in Lieu of Taxes) paid to Cambridge.

In [30]:
import numpy as np
import pandas as pd
from scipy import stats
import pickle

In [3]:
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [4]:
plotly.tools.set_credentials_file(username='hangulu', api_key='78VR3oagCeoHkdYiKB4b')

In [5]:
expander = pd.read_excel("towngown_data.xlsx", header=1)
expander = expander.round(0)
expander.head()

,Year,Student Population,Tax Exempt Property (SF),Tax Exempt Property (A),Taxable Property (SF),Taxable Property (A),Payments to City
0,2000,20529,11800000,271.0,900000,21.0,8467151
1,2001,20975,11500000,264.0,995100,23.0,8751365
2,2002,20791,11640000,267.0,1130000,26.0,10366585
3,2003,21273,11640000,267.0,1300000,30.0,10233641
4,2004,21201,8450640,194.0,1263240,29.0,12083442


In [10]:
# Growth of student population
years = pd.to_numeric(expander['Year']).as_matrix()
stud_pop = pd.to_numeric(expander['Student Population']).as_matrix()

slope, intercept, r_value, p_value, std_err = stats.linregress(years, stud_pop)
studpop_fit = slope * years + intercept

data = go.Data([
        go.Scatter(
            x = years,
            y = stud_pop
        ),
        go.Scatter(
            x = years,
            y = studpop_fit,
            mode='lines',
            marker=go.Marker(color='rgb(255, 63, 52)'),
            name='Fit'
            )])
layout = go.Layout(
            title = "The Growth of Harvard's Student Population",
            yaxis=dict(
                title='Number of Students'),
            xaxis=dict(
                title='Year')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='student_population')

In [14]:
# Growth of property
years = np.arange(2003, 2017)
post_2000 = expander[expander['Year'] > 2002]
props = pd.to_numeric(post_2000['Taxable Property (A)']).as_matrix()

slope, intercept, r_value, p_value, std_err = stats.linregress(years, props)
props_fit = slope * years + intercept

trace0 = go.Scatter(
            x = expander['Year'],
            y = expander['Tax Exempt Property (A)'],
            name='Tax Exempt'
        )
trace1 = go.Scatter(
            x = expander['Year'],
            y = expander['Taxable Property (A)'],
            name='Taxable',
            yaxis='y2'
        )
fit0 = go.Scatter(
            x = years,
            y = props_fit,
            mode='lines',
            marker=go.Marker(color='rgb(76, 209, 55)'),
            name='Taxable Fit',
            yaxis='y2'
        )

data = [trace0, trace1, fit0]

layout = go.Layout(
            title = "The Growth of Harvard's Property in Cambridge Since 2000",
            yaxis=dict(
                title='Tax Exempt Property (Acres)'
            ),
            yaxis2=dict(
                title='Taxable Property (Acres)',
                overlaying='y',
                side='right'
            ),
            xaxis=dict(
                title='Year')
        )

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='property_expansion')

In [28]:
# Change in payments to Cambridge

years = pd.to_numeric(expander['Year']).as_matrix()
pay_city = pd.to_numeric(expander['Payments to City']).as_matrix()

poly_fit = np.polyfit(years, pay_city, 3)
f = np.poly1d(poly_fit)

data = go.Data([
        go.Scatter(
            x = years,
            y = pay_city,
            name='Data'
        ),
        go.Scatter(
            x = years,
            y = f(years),
            mode='lines',
            marker=go.Marker(color='rgb(255, 63, 52)'),
            name='Fit'
            )])
layout = go.Layout(
            title = "How Harvard Pays Cambridge Annually",
            yaxis=dict(
                title='Payments to Cambridge (USD)'),
            xaxis=dict(
                title='Year')
)



fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='pay_city')

Save the data for use in another notebook.

In [31]:
with open('VariableData/expander.pickle', 'wb') as f:
    pickle.dump(expander, f)